In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.signal import savgol_filter

from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold, cross_val_predict, train_test_split
import piplite
await piplite.install('seaborn')
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


In [ ]:
data = pd.read_csv("data.csv")
data         

In [ ]:
data.drop('id', axis=1, inplace=True)
data['diagnosis'] = data['diagnosis'].replace('M', 0)
data['diagnosis'] = data['diagnosis'].replace('B', 1)
data.head()

In [ ]:
Malignant, Benign = data.diagnosis.value_counts()
Malignant, Benign

In [ ]:
data.drop(["perimeter_mean","perimeter_se","perimeter_worst"], axis = "columns", inplace = True)
data.drop(["Radius_mean","radius_se", "radius_worst"], axis = "columns",inplace=True)
data.drop(["symmetry_se","texture_se","smoothness_se","fractal_dimension_se"],axis="columns",inplace=True)
data.drop("fractal_dimension_mean",axis="columns",inplace=True)
data.drop("Texture_mean",axis="columns", inplace=True)
data.drop(["area_mean","compactness_worst"], axis="columns",inplace=True)
plt.figure(figsize=(25,20))
sns.heatmap(data=data.corr(),annot=True)

In [ ]:
x = data.drop(columns=['diagnosis'])
y = data.diagnosis

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size =0.25, random_state=42)
pls = PLSRegression()
pred = pls.fit(x_train,y_train)
y_pred=pls.predict(x_test)

In [ ]:
train_score= pls.score(x_train, y_train)
test_score = pls.score(x_test, y_test)
print("training score:", train_score, "\ntest score:", test_score)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.ensemble import RandomForestClassifier
rf  = RandomForestClassifier() # PLS does not have the predict_proba attribute so random forest was used 
pred = rf.fit(x_train, y_train)
y_scores = rf.predict_proba(x_test)
fpr, tpr, thresholds = roc_curve(y_test, y_scores[:, 1])
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr, label="ROC curve (area = %0.2f)" % roc_auc)
plt.plot([0,1],[0,1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis 
# Since PLS uses regression, confusion matrix can't be found, LDA was used to find the confusion matrix
lda = LinearDiscriminantAnalysis()
pred = lda.fit(x_train,y_train)
y_pred=lda.predict(x_test)

cm = confusion_matrix(y_test, y_pred)
accuracy_score(y_test, y_pred)
s = sns.heatmap(cm, annot=True)
s.set(xlabel='Predicted', ylabel='True')